# MicroGrad

In [54]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [55]:
from graphviz import Digraph

def trace(root):
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child,v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir':'LR'})
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    dot.node(name=uid, label="{ %s | data %.4f | grad%.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      dot.node(name=uid + n._op, label = n._op)
      dot.edge(uid + n._op, uid)
  for n1, n2 in edges:
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)
  return dot



In [62]:
class Value:
  def __init__(self, data, _children=(), _op='', label = ''):
    self.data = data
    self._prev = set(_children)
    self.grad = 0.0
    self._backward = lambda: None
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data = {self.data})"

  def __radd__(self, other):
    return self + other

  def __rmul__(self, other):
    return self * other

  def __truediv__(self, other):
    return self * (other ** -1)

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int and float powers for now"
    out = Value(self.data ** other, (self,), f'**{other}')
    def _backward():
      self.grad += out.grad * (other * self.data ** (other - 1))
    out._backward = _backward
    return out

  def __neg__(self):
    return self * -1

  def __sub__(self, other):
    return self + (-other)

  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self,), 'exp')
    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward
    return out

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    return out

  def backward(self):
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    self.grad = 1
    for node in reversed(topo):
      node._backward()

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out

  def tanh(self):
    n = self.data
    t = (math.exp(2*n)-1)/(math.exp(2*n)+1)
    out = Value(t, (self,), 'tanh')

    def _backward():
      self.grad += (1-t**2) * out.grad
    out._backward = _backward
    return out


In [76]:
import random

class Neuron:

  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))

  def __call__(self, x):
    act = sum(((wi * xi) for wi, xi in zip(self.w, x)), Value(0))
    act = act + self.b
    return act.tanh()

  def parameters(self):
    return self.w + [self.b]

class Layer:

  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:

  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [253]:
x = [2.0, 3.0, 4.0]
n = MLP(3,[4,4,1])
n(x)

Value(data = -0.7289057212708597)

In [254]:
#Data for Binary Classifier
xs = [
    [2.0, 3,-1],
    [3,-1,0.5],
    [0.5,1,1],
    [1,1,-1],
]
ys = [1,-1,-1,1]

In [255]:
for k in range (100):
  ypred = [n(x) for x in xs]
  loss = sum((yout-ygt)**2 for ygt, yout in zip(ys, ypred))

  for p in n.parameters():
    p.grad = 0
  loss.backward()

  for p in n.parameters():
    p.data -= 0.1 * p.grad
  print(k+1, loss.data)


1 7.755378966750016
2 6.811252148787441
3 4.938941537833793
4 4.5451503182114505
5 4.351257812797262
6 4.164030582162215
7 3.8828712795587483
8 3.50952763998587
9 2.3385637506589534
10 1.9100103034053812
11 0.7552503249379394
12 0.05476136015249134
13 0.046528024569700455
14 0.040585268068767576
15 0.03605024388661532
16 0.032454750860291016
17 0.02952352980327776
18 0.02708234791114493
19 0.025014640178333984
20 0.02323900009443965
21 0.021696595534295864
22 0.020343706021216497
23 0.01914707948714557
24 0.018080926098501794
25 0.017124905793529575
26 0.016262742139008003
27 0.015481243877421932
28 0.01476959936003985
29 0.014118858139963827
30 0.013521543713121592
31 0.012971359923473171
32 0.012462965408311405
33 0.011991798229883232
34 0.01155393803841755
35 0.011145996656362782
36 0.010765030432284488
37 0.010408469445013334
38 0.010074059876480624
39 0.00975981676793731
40 0.009463985031058043
41 0.0091850070721581
42 0.008921495752177076
43 0.008672211680582608
44 0.008436044051

In [256]:
ypred

[Value(data = 0.9744904623132525),
 Value(data = -0.9677435883536426),
 Value(data = -0.9740329583656596),
 Value(data = 0.9700097696110207)]